<center>
<b><font size=6>Industry 4.0 의 중심, BigData</font></b>
</center>

<div align='right'><font size=2 color='gray'>Data Processing Based Python @ <font color='blue'><a href='https://www.facebook.com/jskim.kr'>FB / jskim.kr</a></font>, 김진수</font></div>
<hr>

# 공공데이터 활용

## Case5. 카페주소와 지도위치(위도,경도)값 구하기
 https://www.data.go.kr/data/15086876/fileData.do

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import seaborn as sns

from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

%matplotlib inline
plt.rcParams["figure.figsize"] = [12,6]

### Step1. 데이터 준비
> DATA : 공공데이터의 인천 중구 커피숍 파일
> - Link : https://www.data.go.kr/data/15086876/fileData.do



In [2]:
# %ls data\*.csv
file_name = './data/public_incheon_cafe_2021.csv'

### Step2. 데이터 확인

In [3]:
# csv파일 불러오기
# df = pd.read_csv(file_name, encoding='utf-8')  # utf-8, euc-kr, cp949
df = pd.read_csv(file_name, encoding='euc-kr')
df.head()

,연번,사업장명,소재지지번주소,소재지도로명주소,데이터기준일자
0,1,Cafe Arancia,인천광역시 중구 내동 212-22 1층,인천광역시 중구 우현로49번길 10-5 1층,2021-08-05
1,2,디저트카페 아네모오네,인천광역시 중구 송학동3가 7-26,인천광역시 중구 신포로35번길 10-1,2021-08-05
2,3,관동오리진,인천광역시 중구 관동1가 14-3,인천광역시 중구 신포로27번길 96,2021-08-05
3,4,카페맑음,인천광역시 중구 신포동 48-3,인천광역시 중구 신포로 32-21,2021-08-05
4,5,팟알,인천광역시 중구 관동1가 17,인천광역시 중구 신포로27번길 96-2,2021-08-05


### Step3. 필요한 주소만 남겨놓기

In [4]:
col_addr1 = df['소재지지번주소']
col_addr1[0]

'인천광역시 중구 내동 212-22 1층'

In [5]:
col_addr2 = df['소재지도로명주소']
col_addr2[0]

'인천광역시 중구 우현로49번길 10-5 1층'

In [6]:
len(col_addr1), len(col_addr2)

(80, 80)

In [7]:
# 주소 데이터 깔끔하게 다듬기
arr_address = list()
for idx in range(len(col_addr2)):
    tmp = col_addr2[idx].split()
    address = " ".join(tmp[0:4])
    arr_address.append(address)
    

In [8]:
arr_address[:5]

['인천광역시 중구 우현로49번길 10-5',
 '인천광역시 중구 신포로35번길 10-1',
 '인천광역시 중구 신포로27번길 96',
 '인천광역시 중구 신포로 32-21',
 '인천광역시 중구 신포로27번길 96-2']

### Step4. 위경도 변환하기
> 
> - geopy 라이브러리를 import 
> - 도로명주소 위도 경도 값으로 바꿔주기 

In [9]:
from geopy.geocoders import Nominatim

In [10]:
# 도로명주소 위도 경도 값으로 바꿔주기 
geo_local = Nominatim(user_agent='South Korea')

In [11]:
# 위도, 경도 반환하는 함수
def geocoding(address):
    '''
    geocoding함수 안에 주소를 넣어주면 그 주소에 맞는 위도, 경도를
    리스트로 만들어 [위도, 경도]로 반환
    '''
    geo = geo_local.geocode(address)
    # print('{} : {}'.format(address, geo))  
    
    geo_loc= (geo.latitude, geo.longitude) if geo else (None, None)
        
    return geo_loc

In [12]:
address = arr_address[0]
address

'인천광역시 중구 우현로49번길 10-5'

In [13]:
geo = geo_local.geocode(address)
geo

Location(우현로49번길, 신포국제시장, 신포동, 중구, 인천, 22321, 대한민국, (37.4716422, 126.6273599, 0.0))

In [14]:
type(geo)

geopy.location.Location

In [15]:
dir(geo)[-10:]

['_address',
 '_point',
 '_raw',
 '_tuple',
 'address',
 'altitude',
 'latitude',
 'longitude',
 'point',
 'raw']

In [16]:
geo.address

'우현로49번길, 신포국제시장, 신포동, 중구, 인천, 22321, 대한민국'

In [17]:
geo.altitude

0.0

In [18]:
geo.latitude

37.4716422

In [19]:
geo.longitude

126.6273599

In [20]:
geo.point

Point(37.4716422, 126.6273599, 0.0)

In [21]:
geo.raw

{'place_id': 206054857,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 472437323,
 'boundingbox': ['37.4715467', '37.4717037', '126.6264296', '126.6285951'],
 'lat': '37.4716422',
 'lon': '126.6273599',
 'display_name': '우현로49번길, 신포국제시장, 신포동, 중구, 인천, 22321, 대한민국',
 'class': 'highway',
 'type': 'service',
 'importance': 0.295}

In [22]:
# from tqdm import tqdm_notebook
from tqdm.notebook import tqdm
import time

In [23]:
arr_lat = list()
arr_lon = list()

for idx in tqdm(df.index):
    # col_addr = df['소재지지번주소']
    col_addr = df['소재지도로명주소']
    # tmp = col_addr[idx].split()
    # address = " ".join(tmp[0:4])
    address = " ".join(col_addr[idx].split()[0:4])
    
    geo_lat, geo_lon =  geocoding(address)
    
    arr_lat.append(geo_lat)
    arr_lon.append(geo_lon)
    

  0%|          | 0/80 [00:00<?, ?it/s]

In [24]:
geo_loc = list(zip(arr_lat, arr_lon))
geo_loc[:5]

[(37.4716422, 126.6273599),
 (37.4733945, 126.6231442),
 (37.4727937, 126.6225932),
 (37.4709967, 126.6242367),
 (37.4727937, 126.6225932)]

In [25]:
df.head()

,연번,사업장명,소재지지번주소,소재지도로명주소,데이터기준일자
0,1,Cafe Arancia,인천광역시 중구 내동 212-22 1층,인천광역시 중구 우현로49번길 10-5 1층,2021-08-05
1,2,디저트카페 아네모오네,인천광역시 중구 송학동3가 7-26,인천광역시 중구 신포로35번길 10-1,2021-08-05
2,3,관동오리진,인천광역시 중구 관동1가 14-3,인천광역시 중구 신포로27번길 96,2021-08-05
3,4,카페맑음,인천광역시 중구 신포동 48-3,인천광역시 중구 신포로 32-21,2021-08-05
4,5,팟알,인천광역시 중구 관동1가 17,인천광역시 중구 신포로27번길 96-2,2021-08-05


In [26]:
file_name = './data/public_incheon_cafe_2021_with_geo.csv'

In [27]:
# Dataframe만들기
geo_df = pd.DataFrame({
    '카페이름': df['사업장명'],
    '카페주소':df['소재지도로명주소'],
    # '지번주소':df['소재지지번주소'],
    '위도':arr_lat,
    '경도':arr_lon})

geo_df.head()

,카페이름,카페주소,위도,경도
0,Cafe Arancia,인천광역시 중구 우현로49번길 10-5 1층,37.471642,126.627360
1,디저트카페 아네모오네,인천광역시 중구 신포로35번길 10-1,37.473394,126.623144
2,관동오리진,인천광역시 중구 신포로27번길 96,37.472794,126.622593
3,카페맑음,인천광역시 중구 신포로 32-21,37.470997,126.624237
4,팟알,인천광역시 중구 신포로27번길 96-2,37.472794,126.622593


In [28]:
#df저장
geo_df.to_csv(file_name)

In [29]:
chk_geo_df = pd.read_csv(file_name)
chk_geo_df.head()

,Unnamed: 0,카페이름,카페주소,위도,경도
0,0,Cafe Arancia,인천광역시 중구 우현로49번길 10-5 1층,37.471642,126.627360
1,1,디저트카페 아네모오네,인천광역시 중구 신포로35번길 10-1,37.473394,126.623144
2,2,관동오리진,인천광역시 중구 신포로27번길 96,37.472794,126.622593
3,3,카페맑음,인천광역시 중구 신포로 32-21,37.470997,126.624237
4,4,팟알,인천광역시 중구 신포로27번길 96-2,37.472794,126.622593


In [30]:
chk_geo_df.columns

Index(['Unnamed: 0', '카페이름', '카페주소', '위도', '경도'], dtype='object')

In [31]:
del chk_geo_df[chk_geo_df.columns[0]]

In [32]:
chk_geo_df.tail()

,카페이름,카페주소,위도,경도
75,낭만카페,인천광역시 중구 우현로39번길 12,37.470894,126.625950
76,또바기커피,인천광역시 중구 우현로45번길 25,37.471257,126.627222
77,카페와비사비,인천광역시 중구 신포로31번길 51-1,37.472425,126.624138
78,카페파구스,인천광역시 중구 개항로53번길 21,37.472645,126.628267
79,이삭커피숍,인천광역시 중구 우현로 12,37.472768,126.630054


<hr>
<marquee><font size=3 color='brown'>The BigpyCraft find the information to design valuable society with Technology & Craft.</font></marquee>
<div align='right'><font size=2 color='gray'> &lt; The End &gt; </font></div>